In [1]:
from itertools import combinations
import numpy as np
#new imports
import numpy as np
import random

## Simple Test Problem

In [2]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('lab2/test_problem.npy')

## Common tests

In [3]:
problem = np.load('lab2/problem_r2_100.npy')

In [4]:
# Negative values?
np.any(problem < 0)

np.True_

In [5]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

False

In [6]:
# Symmetric matrix?
np.allclose(problem, problem.T)

False

In [7]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

False

## Implementation

In [ ]:
problem = np.load('lab2/problem_r1_100.npy')

#distance calculator, fitness of the solution
def distance_calc(solution, problem_matrix):
    distance = problem_matrix[solution[-1]][solution[0]]
    for i in range(len(solution) - 1):
        distance += problem_matrix[solution[i]][solution[i+1]]
    return distance

#greedy solution, going to the closest point from the present one
def nearest_city(problem_matrix, start=0):
    n = len(problem_matrix)
    not_visited = set(range(n))
    solution = [start]
    not_visited.remove(start)
    
    while not_visited:
        current = solution[-1]
        next_city = min(not_visited, key=lambda city: problem_matrix[current][city])
        solution.append(next_city)
        not_visited.remove(next_city)
    return solution

greedy_solution = nearest_city(problem)
greedy_cost = distance_calc(greedy_solution, problem)
print(f"Greedy solution distance: {greedy_cost:.2f}")

#GA solution

#creates a random solution
def new_individual(n_cities):
    individual = list(range(n_cities))
    random.shuffle(individual)
    return individual

#creates the population
def new_population(pop_size, n_cities):
    return [new_individual(n_cities) for _ in range(pop_size)]

#select the best among random individuals
def parent_tournament(population, fitnesses, k):
    best_idx = None
    for _ in range(k):
        idx = random.randrange(len(population))
        if best_idx is None or fitnesses[idx] < fitnesses[best_idx]:
            best_idx = idx
    return population[best_idx]


def crossover(parent1, parent2):
    #random segment taken by parent 1
    n = len(parent1)
    child = [None] * n
    start, end = sorted(random.sample(range(n), 2))
    child[start:end+1] = parent1[start:end+1]
    
    #remaining part of the solution taken by parent 2
    p2_idx = 0
    for i in range(n):
        if child[i] is None:
            #skips the elements already present in child
            while parent2[p2_idx] in child:
                p2_idx += 1
            child[i] = parent2[p2_idx]
            p2_idx += 1
    return child

#mutation that inverts element in a segment, solution validity garanteed
def mutation(individual):
    start, end = sorted(random.sample(range(len(individual)), 2))
    segment = individual[start:end+1]
    segment.reverse()
    individual[start:end+1] = segment
    return individual

#starts with the greedy solution
def genetic_alg(problem_matrix, pop_size, n_generations, crossover_rate, mutation_rate, k_tourn, n_elit, greedy_solution):

    n_cities = problem_matrix.shape[0]
    population = new_population(pop_size, n_cities)
    population[0] = greedy_solution[:] #inserting greedy solution in the population    
    best_solution = greedy_solution[:]
    best_fitness = distance_calc(best_solution, problem_matrix)
    
    print(f"Initial greedy fitness: {best_fitness:.2f}")

    for gen in range(n_generations):
        fitnesses = [distance_calc(ind, problem_matrix) for ind in population]
        current_best_idx = np.argmin(fitnesses)
        current_best_fitness = fitnesses[current_best_idx]
        
        #new best fitness
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            best_solution = population[current_best_idx]
            print(f"Generation {gen:4}: New best fitness = {best_fitness:.2f}")
        elif gen % 1000 == 0: # print every 1000 generations
            print(f"Generation {gen:4}: (fitness: {best_fitness:.2f})")
            
        next_population = []
        
        #takes n_elit best individuals
        sorted_indices = np.argsort(fitnesses)
        for i in range(n_elit):
            next_population.append(population[sorted_indices[i]])
            
        #fills next population
        while len(next_population) < pop_size:
            #takes the best individual among random k_tourn individuals
            p1 = parent_tournament(population, fitnesses, k_tourn)
            p2 = parent_tournament(population, fitnesses, k_tourn)
            
            #crossover and mutation
            if random.random() < crossover_rate:
                child = crossover(p1, p2)
            else:
                child = p1[:]
            if random.random() < mutation_rate:
                mutation(child)

            next_population.append(child)
        
        population = next_population
    return best_solution, best_fitness

#main
#parameters chosen with a prior tuning
POP_SIZE = 300        
GENERATIONS = 10000   
CROSSOVER_RATE = 0.9  
MUTATION_RATE = 0.8   
K_TOURN = 5      
N_ELIT = 1         

#{'pop_size': 300, 'mutation_rate': 0.8, 'k_tourn': 5, 'crossover_rate': 0.9, 'n_elit': 1}
#{'pop_size': 300, 'mutation_rate': 0.6, 'k_tourn': 3, 'crossover_rate': 0.8, 'n_elit': 2}

best_solution, best_fitness = genetic_alg(
    problem_matrix=problem,
    pop_size=POP_SIZE,
    n_generations=GENERATIONS,
    crossover_rate=CROSSOVER_RATE,
    mutation_rate=MUTATION_RATE,
    k_tourn=K_TOURN,
    n_elit=N_ELIT,
    greedy_solution=greedy_solution
)

if best_fitness < greedy_cost:
    print(f"GA improved the greedy by {greedy_cost - best_fitness} points")
else:
    print("GA did not improve the greedy solution")

print(f"Best solution: {best_solution}\n Best fitness: {best_fitness}")

Greedy solution distance: {greedy_cost:.2f}
Initial greedy fitness: 908.81
Generation    0: (fitness: 908.81)
Generation   16: New best fitness = 899.98
Generation   47: New best fitness = 891.37
Generation   49: New best fitness = 864.22
Generation   75: New best fitness = 854.25
Generation   85: New best fitness = 841.08
Generation  100: New best fitness = 837.43
Generation  113: New best fitness = 836.38
Generation  123: New best fitness = 832.33
Generation  135: New best fitness = 831.28
Generation  158: New best fitness = 829.76
Generation  231: New best fitness = 823.52
Generation  246: New best fitness = 822.67
Generation  267: New best fitness = 822.41
Generation  275: New best fitness = 821.82
Generation  277: New best fitness = 821.55
Generation  318: New best fitness = 814.26
Generation  362: New best fitness = 808.77
Generation  365: New best fitness = 802.32
Generation  383: New best fitness = 801.50
Generation  386: New best fitness = 799.29
Generation  398: New best fitn

KeyboardInterrupt: 